# Pooler AI Training (Fixed Version)
Training setup with memory management

In [ ]:
# Install required packages
!pip install torch transformers google-cloud-storage tqdm psutil

In [ ]:
# Clean working directory and download files
!rm -rf /kaggle/working/*  # Clean directory first
!mkdir -p /kaggle/working/pooler
!cd /kaggle/working && curl -L -o pooler_files.zip https://github.com/Black5681/Ai-demo/raw/main/pooler_kaggle_files.zip
!cd /kaggle/working && unzip -o pooler_files.zip  # Force overwrite
!ls -la /kaggle/working  # Check the extracted files

In [ ]:
# Memory management setup
import gc
import torch
import psutil
import os

def setup_memory_management():
    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        
        # Get GPU info
        gpu_props = torch.cuda.get_device_properties(0)
        total_gpu_mem = gpu_props.total_memory / 1e9
        
        # Set conservative memory limits
        if total_gpu_mem > 14:  # T4 GPU (16GB)
            mem_fraction = 0.6  # Use 60% of GPU memory
            batch_size = 8
        else:  # Smaller GPU
            mem_fraction = 0.5  # Use 50% of GPU memory
            batch_size = 4
            
        # Apply memory limits
        torch.cuda.set_per_process_memory_fraction(mem_fraction)
        
        print(f"GPU: {gpu_props.name}")
        print(f"Total GPU Memory: {total_gpu_mem:.2f} GB")
        print(f"Using {mem_fraction*100}% of GPU memory")
        print(f"Batch size set to: {batch_size}")
        
        return batch_size
    else:
        print("No GPU available, using CPU")
        return 2  # Small batch size for CPU

# Set environment variables for PyTorch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# Setup memory management
BATCH_SIZE = setup_memory_management()

In [ ]:
# Run training with memory-optimized settings
!BATCH_SIZE={BATCH_SIZE} PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128 python train_lowmem.py